## Set-Up

In [2]:
import numpy as np
import pandas as pd
from sqlalchemy import create_engine, inspect, func

In [3]:
# Create an engine for the `wine_db.sqlite` database
engine = create_engine("sqlite:///Resources/wine_data.db")

In [4]:
inspector = inspect(engine)
inspector.get_table_names()

['winequalityRed', 'winequalityWhite']

In [5]:
red_wine = engine.execute('SELECT * FROM winequalityRed').fetchall()

In [6]:
wine_red_df = pd.DataFrame(red_wine, columns=["wine_id","fixed acidity","volatile acidity", "citric acid",
                                "residual sugar","chlorides","free sulfur dioxide","total sulfur dioxide",
                                "density","pH","sulphates","alcohol","quality"])

In [10]:
# Separate feature variables and target variable
X = wine_red_df.drop(['quality','wine_id'], axis = 1)
y = wine_red_df['quality']

In [12]:
# Normalize feature variables
from sklearn.preprocessing import StandardScaler
X_features = X
scalar = StandardScaler().fit(X)
X = StandardScaler().fit_transform(X)

In [13]:
# Splitting the data
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=.25, random_state=0)

In [14]:
from sklearn.metrics import classification_report
import xgboost as xgb
model = xgb.XGBClassifier(random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.77      0.82      0.79       185
           1       0.74      0.66      0.70       170
           2       0.63      0.73      0.68        45

    accuracy                           0.74       400
   macro avg       0.72      0.74      0.72       400
weighted avg       0.74      0.74      0.74       400



In [15]:
import pickle
pkl_filename = "red_model.pkl"
with open(pkl_filename, 'wb') as file:
    pickle.dump(model,file)

scaler = StandardScaler().fit(X)
pkl_scaled_filename = 'scaler_red.pkl'
with open(pkl_scaled_filename, 'wb') as file:
    pickle.dump(scaler,file)

In [19]:
# Function to Load and run model with input scores

def red_model(user_input_array):
    import pickle
    #Load Model
    with open('red_model.pkl', 'rb') as file:
        red_model = pickle.load(file)
    with open('scaler_red.pkl', 'rb') as file:
        scaler = pickle.load(file)

    X = user_input_array.reshape(1,-1)
    X_scaled = scaler.transform(X)

    # Calculate Wine quality
    y_pred_red = red_model.predict(X_scaled)

    return y_pred_red

In [20]:
test = X_test[1]
answer = y[1]
print(test, answer)

[-0.1261883   1.63225386 -1.39147228  1.10763304  0.16011403 -1.03997712
 -0.98731203  0.9504846   0.3167512  -0.75624575 -0.77251161] 0


In [21]:
red_model(test)

array([0])